# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas as hv
import pandas as pd
import requests
import cartopy.crs as ccrs


# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,-1.03,49,58,0.92,GL,1697776863
1,1,genhe,50.7833,121.5167,1.56,29,42,5.41,CN,1697776920
2,2,mount carmel,38.4109,-87.7614,14.98,70,66,2.46,US,1697776921
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.33,98,100,10.37,SH,1697776869
4,4,sambava,-14.2667,50.1667,25.83,70,22,1.32,MG,1697776888


In [37]:
# Configure the map plot
# Create a Points object using hv.Points with the DataFrame and specifying the latitude and longitude columns
# The size of the point should be the humidity in each city.
map_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map
map_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# .loc subset based on my ideal conditions, remove null values
subset_df = city_data_df.loc[(city_data_df["Max Temp"] > 21.1) 
                             & (city_data_df["Max Temp"] < 35)
                             & (city_data_df["Wind Speed"] < 8)
                             & (city_data_df["Humidity"] < 75)
                             & (city_data_df["Cloudiness"] <= 45), :]
subset_df = subset_df.reset_index()
# check for null and drop any null columns
subset_df.dropna(axis=0)
subset_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,4,4,sambava,-14.2667,50.1667,25.83,70,22,1.32,MG,1697776888
1,7,7,hadibu,12.6500,54.0333,27.76,71,19,3.71,YE,1697776887
2,18,18,ondjiva,-17.0667,15.7333,21.29,20,0,2.32,AO,1697776924
3,21,21,rongelap,11.1635,166.8941,28.55,73,40,5.64,MH,1697776924
4,28,28,tura,25.5198,90.2201,27.46,59,0,0.17,IN,1697776873


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
col_selection = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
# using [[]] creates an issue of chain indexing, so we use .loc again
hotel_df = subset_df.loc[:, col_selection].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sambava,MG,-14.2667,50.1667,70,
1,hadibu,YE,12.6500,54.0333,71,
2,ondjiva,AO,-17.0667,15.7333,20,
3,rongelap,MH,11.1635,166.8941,73,
4,tura,IN,25.5198,90.2201,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories":"accommodation.hotel",
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
sambava - nearest hotel: Royal Sava Hôtel
hadibu - nearest hotel: No hotel found
ondjiva - nearest hotel: Águia Verde Hotel
rongelap - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
mildura - nearest hotel: Chaffey Motor Inn
lithgow - nearest hotel: Grand Central Hotel
west island - nearest hotel: Cocos Village Bungalows
port augusta - nearest hotel: Hotel Commonwealth
howrah - nearest hotel: Sun India Guest House
alofi - nearest hotel: Matavai Resort
conceicao do araguaia - nearest hotel: No hotel found
patterson - nearest hotel: Hampton Inn & Suites
acapulco de juarez - nearest hotel: Hotel del Valle
saipan - nearest hotel: Chalan Kanoa Beach Hotel
paradip garh - nearest hotel: Hotel Sea Pearl
novo aripuana - nearest hotel: Hotel Novo Aripuanã
broken hill - nearest hotel: Ibis Styles
gebeit - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة
kuta - nearest hotel: Ida Hotel Bali
livingstone - nearest hotel: New Fairmount Hotel an

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sambava,MG,-14.2667,50.1667,70,Royal Sava Hôtel
1,hadibu,YE,12.6500,54.0333,71,No hotel found
2,ondjiva,AO,-17.0667,15.7333,20,Águia Verde Hotel
3,rongelap,MH,11.1635,166.8941,73,No hotel found
4,tura,IN,25.5198,90.2201,59,No hotel found
...,...,...,...,...,...,...
75,biltine,TD,14.5333,20.9167,21,فندق ثلاث نجوم بلتن
76,cobija,BO,-11.0267,-68.7692,57,Hotel El Cristo
77,konarka,IN,19.9000,86.1167,60,Hotel Sun Village
78,mulayjah,SA,27.2710,48.4242,18,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    scale = 1.5,
    color = "City",
    hover_cols = ["Country","Hotel Name"]
)

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)